In [63]:
import joblib
from tensorflow.keras.models import load_model
import numpy as np

xgb_model = joblib.load("xgb.pkl")
rf_model = joblib.load("rf_clf.pkl")
fnn_model = load_model("fnn.h5")
rnn_model = load_model("model.h5")
scaler = joblib.load('scaler.pkl')

def dealer_bust_probability(dealer_upcard, player_hand_total):
    base_bust_probabilities = {
        2: 0.35,  
        3: 0.37, 
        4: 0.40, 
        5: 0.42,  
        6: 0.42,  
        7: 0.26, 
        8: 0.24,  
        9: 0.23,  
        10: 0.17,
        11: 0.14 
    }
    base_probability = base_bust_probabilities.get(dealer_upcard, 0)
    if player_hand_total >= 18:
        if dealer_upcard in [4, 5, 6]:
            adjustment_factor = 0.15 
        elif dealer_upcard in [7, 8, 9]:
            adjustment_factor = 0.10  
        elif dealer_upcard in [10, 11]:
            adjustment_factor = 0.05  
        else:
            adjustment_factor = 0.10  
    elif player_hand_total >= 14:
        if dealer_upcard in [4, 5, 6]:
            adjustment_factor = 0.08  
        elif dealer_upcard in [7, 8, 9]:
            adjustment_factor = 0.05 
        elif dealer_upcard in [10, 11]:
            adjustment_factor = 0.03  
        else:
            adjustment_factor = 0.05  
    elif player_hand_total >= 12:
        if dealer_upcard in [4, 5, 6]:
            adjustment_factor = 0.05  
        elif dealer_upcard in [7, 8, 9]:
            adjustment_factor = 0.03  
        elif dealer_upcard in [10, 11]:
            adjustment_factor = 0.01  
        else:
            adjustment_factor = 0.03 
    else:
        adjustment_factor = 0.0
    adjusted_probability = base_probability + adjustment_factor
    adjusted_probability = min(max(adjusted_probability, 0), 1)

    return adjusted_probability

def player_bust_probability(player_hand_total):
    # Bust probabilities based on the player's hand total
    bust_probabilities = {
        21: 1.00,
        20: 0.92,  
        19: 0.85, 
        18: 0.77, 
        17: 0.69, 
        16: 0.62,  
        15: 0.58, 
        14: 0.56,  
        13: 0.39, 
        12: 0.31,
    }
    if(player_hand_total<12):
        return 0
    else:
        return bust_probabilities.get(player_hand_total)
    
def hand_type(hand):
    if 11 in hand and sum(hand) == 21:  # Ace as 11
        return 3
    elif 11 in hand and sum(hand) < 21:  # Ace as 11
        return 1
    elif hand[0] == hand[1]:  # Pair
        return 2
    else:
        return 0

def should_double_down(row):
    if row['current_hand_value'] == 11:  
        return 1 
    elif row['current_hand_value'] == 10:  
        return 1 if row['dealer_up'] in range(2, 10) else 0  
    elif row['current_hand_value'] == 9: 
        return 1 if row['dealer_up'] in range(3, 7) else 0  
    
    # Soft Hands (with Ace)
    elif row['hand_type'] == 1 and row['current_hand_value'] == 18:  
        return 1 if row['dealer_up'] == 6 else 0  
    elif row['hand_type'] == 1 and row['current_hand_value'] == 17:  
        return 1 if row['dealer_up'] in range(3, 7) else 0  
    elif row['hand_type'] == 1 and row['current_hand_value'] == 16: 
        return 1 if row['dealer_up'] in range(4, 7) else 0  
    elif row['hand_type'] == 1 and row['current_hand_value'] == 15: 
        return 1 if row['dealer_up'] in range(4, 7) else 0  
    elif row['hand_type'] == 1 and row['current_hand_value'] == 14: 
        return 1 if row['dealer_up'] in range(5, 7) else 0 
    elif row['hand_type'] == 1 and row['current_hand_value'] == 13: 
        return 1 if row['dealer_up'] in range(5, 7) else 0 

    # Pairs
    elif row['hand_type'] == 2 and row['current_hand_value'] == 5: 
        return 1 if row['dealer_up'] in range(2, 10) else 0 
    elif row['hand_type'] == 2 and row['current_hand_value'] == 9: 
        return 1 if row['dealer_up'] in range(2, 10) and row['dealer_up'] != 7 else 0  

    return 0  

def predict_action(features, actual_action=None):
    # Extract dealer upcard and player hand total for dealer_bust_probability
    dealer_upcard = features[0]  # First feature
    player_hand_total = features[1]  # Second feature
    dealer_ace = features[2]
    more_than_two_cards = features[3]
    player_hand = features[4]
    
    # Calculate hand type
    calculated_hand_type = hand_type(player_hand)
    
    row = {
        "current_hand_value": player_hand_total,
        "dealer_up": dealer_upcard,
        "hand_type": player_hand
    }
    
    extended_features = [
        dealer_upcard,
        player_hand_total,
        dealer_bust_probability(dealer_upcard, player_hand_total),
        player_bust_probability(player_hand_total),
        calculated_hand_type,  # Use calculated_hand_type here
        dealer_ace,
        more_than_two_cards,
        should_double_down(row),
    ]
    
    # Convert features to numpy array
    raw_features = np.array(extended_features).reshape(1, -1)
    
    # Scale first 4 features and the last feature
    to_scale = np.concatenate([raw_features[:, :4], raw_features[:, -1:]], axis=1)
    scaled_values = scaler.transform(to_scale)

    full_features = np.hstack([
        scaled_values[:, :4],        # Scaled first 4 features
        raw_features[:, 4:-1],      # Unscaled middle features
        scaled_values[:, 4:]        # Scaled last feature
    ])
    
    print(full_features)
    
    # Get predictions from each model
    predictions = {}
    action_mapping = {0: "DOUBLE_DOWN", 1: "HIT", 2: "SPLIT", 3: "STAND", 4: "SURRENDER"}

    # FNN Model
    full_features_fnn = full_features.reshape(1, -1)
    fnn_probabilities = fnn_model.predict(full_features_fnn)[0]  # Get probabilities
    fnn_prediction = np.argmax(fnn_probabilities)
    fnn_confidence = fnn_probabilities[fnn_prediction] * 100  # Confidence percentage
    predictions["FNN"] = (action_mapping[fnn_prediction], fnn_confidence)

    # RNN Model
    full_features_rnn = full_features.reshape(1, 1, -1)
    rnn_probabilities = rnn_model.predict(full_features_rnn)[0]  # Get probabilities
    rnn_prediction = np.argmax(rnn_probabilities)
    rnn_confidence = rnn_probabilities[rnn_prediction] * 100  # Confidence percentage
    predictions["RNN"] = (action_mapping[rnn_prediction], rnn_confidence)

    # XGBoost Model
    xgb_probabilities = xgb_model.predict_proba(full_features)[0]  # Get probabilities
    xgb_prediction = np.argmax(xgb_probabilities)
    xgb_confidence = xgb_probabilities[xgb_prediction] * 100  # Confidence percentage
    predictions["XGBOOST"] = (action_mapping[xgb_prediction], xgb_confidence)

    # Random Forest Model
    rf_probabilities = rf_model.predict_proba(full_features)[0]  # Get probabilities
    rf_prediction = np.argmax(rf_probabilities)
    rf_confidence = rf_probabilities[rf_prediction] * 100  # Confidence percentage
    predictions["RF"] = (action_mapping[rf_prediction], rf_confidence)

    # Print out all predictions with confidence percentages
    result_string = ""
    for model, (prediction, confidence) in predictions.items():
        result_string += f"{model}: {prediction} (Confidence: {confidence:.2f}%)\n"
    
    print(result_string)
    
    return predictions  # If you want to return the predictions dictionary

# Example of how to call the function
predictions = predict_action([6,18,1,0,[9,9]])


[[0.44444444 0.82352941 1.         0.77       2.         1.
  0.         0.        ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


c:\Users\Bruger\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step
FNN: STAND (Confidence: 59.69%)
RNN: SPLIT (Confidence: 99.41%)
XGBOOST: SPLIT (Confidence: 86.54%)
RF: SPLIT (Confidence: 46.00%)



c:\Users\Bruger\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
